In [55]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [70]:
import pandas as pd
import requests
from data_retrieve.fsq_reqs import fetch_fsq_info
import time

In [57]:
query = """
        [out:json][timeout:25];

        area["ISO3166-2"="US-CA"]->.searchArea;
        (
        node["shop"="laundry"](area.searchArea);
        way["shop"="laundry"](area.searchArea);
        relation["shop"="laundry"](area.searchArea);
        node["amenity"="laundry"](area.searchArea);
        way["amenity"="laundry"](area.searchArea);
        relation["amenity"="laundry"](area.searchArea);
        );

        out center;
    """

url = "https://overpass-api.de/api/interpreter"

response = requests.get(url, params={"data": query}, timeout=60)
response.raise_for_status()
data = response.json()

items = []
max_count = 100000
cur_count = 0
print(len(data.get("elements", [])))
print(data.get("elements", [])[0])

1258
{'type': 'node', 'id': 299407735, 'lat': 37.5799133, 'lon': -122.3331493, 'tags': {'created_by': 'Potlatch 0.10b', 'name': 'Holiday Cleaning', 'shop': 'laundry', 'wheelchair': 'yes'}}


In [58]:
locations = data.get("elements", [])

location_data = []
for place in locations:
    lat = place.get("lat") or place.get("center", {}).get("lat")
    lon = place.get("lon") or place.get("center", {}).get("lon")

    tags = place.get("tags", {})

    location_data.append({
        "lat": lat,
        "lon": lon,
        'name': tags.get("name"),
        "email": tags.get("email"),
        "website": tags.get("website"),
        "phone": tags.get("phone")
    })

laundromat_info = pd.DataFrame(location_data)
laundromat_info[laundromat_info["website"].notna()]

,lat,lon,name,email,website,phone
56,35.291402,-120.660052,The Laundromat by Swish & Swirl,None,https://thelaundromatbyswishandswirl.com/,+18057694688
85,37.767590,-122.429766,Wash,us@wash-local.com,http://wash-local.com/,None
135,36.979539,-122.009544,CleanStyle Laundry Lounge,None,https://www.cleanstylelaundry.com/,+1-831-854-7398
140,34.419686,-119.704823,Modern Laundry,None,https://modernlaundry.com/,+1-805-845-0078
149,37.680677,-121.770747,Advantage Laundry,None,http://www.advantagelaundry.com/,None
185,37.650974,-122.491771,Tons of Bubbles Laundromat,None,https://www.tonsofbubbles.com/,None
192,37.662765,-121.870832,Plaza Speed Wash,None,https://www.boswashlaundry.com/pleasanton,None
332,34.155883,-118.792360,TLC Cleaners,None,https://www.tlccleanersca.com/,818-706-8884
377,37.323931,-121.924550,GOGO Laundry,None,https://www.gogolaundry.com/,None
389,34.085963,-118.383077,JB French Dry Cleaners & Laundry,None,https://www.jbfrench.com/,+1-310-652-4343


In [69]:
laundromat_info.iloc[0]

lat               37.579913
lon             -122.333149
name       Holiday Cleaning
email                  None
website                None
phone                  None
Name: 0, dtype: object

In [71]:
def hasResults(row):
    info = fetch_fsq_info(row["lat"], row["lon"])
    time.sleep(0.2)
    return info["results"] != []

In [73]:
mask = laundromat_info[2:].apply(
    lambda row: hasResults(row),
    axis=1
)
filtered_laundromats = laundromat_info[2:][mask]
filtered_laundromats

HTTPError: 429 Client Error: Too Many Requests for url: https://places-api.foursquare.com/places/search?ll=37.0146878%2C-121.5873&radius=200&fsq_category_ids=52f2ab2ebcbc57f1066b8b33&tel_format=NATIONAL&fields=popularity%2Crating%2Cstats%2Cemail%2Ctel%2Cwebsite